In [173]:
from pipeline import PipelineDB
import pandas as pd
import numpy as np
import xgboost as xgb

In [174]:
pp = PipelineDB('l')

In [175]:
raw_data = pp.get_all_inzeraty()
raw_data  = raw_data.replace({-1: None})
raw_data  = raw_data.replace({'': None})

In [178]:
raw_data = raw_data.drop(['id', 'zdroj', 'timestamp'], axis=1)

In [179]:
# prilis malo nenulovych hodnot
raw_data = raw_data.drop(['balkon', 'lodzia', 'verejne_parkovanie','garaz', 'garazove_statie'], axis=1)

In [180]:
# model zatial iba pre bratislavu
raw_data = raw_data.drop(['okres'], axis=1)

In [181]:
# ulicu zatial neviem vyuzit
raw_data = raw_data.drop(['ulica'], axis=1)

In [182]:
# cena za m2 nie vhoda feature, kedze ju neni mozne vypocitat z pozorovani
raw_data = raw_data.drop(['cena_m2'], axis=1)

In [183]:
# drop riadkov kde cena je neznama/dohodu
raw_data = raw_data[raw_data['cena'].notna()]

In [185]:
raw_data = pd.get_dummies(raw_data, columns=['mesto','druh','stav', 'kurenie','energ_cert', 'vytah'], prefix='', prefix_sep='')

In [186]:
y = raw_data['cena']

In [187]:
X = raw_data.drop(['cena'], axis=1)


In [189]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [191]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

In [192]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)


In [193]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

[18:08:50] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [195]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 871746.386977


In [196]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)


[18:12:20] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:12:20] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[18:12:20] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


In [197]:
cv_results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,1.836932e+06,98933.076594,1.881216e+06,210426.855196
1,1.776810e+06,91467.123484,1.849117e+06,204446.534299
2,1.720379e+06,103318.981899,1.813076e+06,200179.245746
3,1.624045e+06,101632.093280,1.743660e+06,222943.290731
4,1.593116e+06,73008.450850,1.723398e+06,242880.504523


In [198]:
print((cv_results["test-rmse-mean"]).tail(1))

49    1.390031e+06
Name: test-rmse-mean, dtype: float64


In [199]:
xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10)

[18:13:20] WARNING: ../src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
